# Momentum Strategy Backtest

This notebook demonstrates the new backtest workflow:
- Configure strategy with a simple Python dict
- Run walk-forward validation
- Auto-generate comprehensive report
- Interactive exploration of results

In [ ]:
# Imports
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from core.backtest_engine import run_walk_forward
from signals.momentum import MomentumSignal
from analysis.report import BacktestReport

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## 1. Load Data

In [ ]:
# Load SPX data
df = pd.read_csv('../Dataset/spx_full_1990_2025.csv', index_col=0, parse_dates=True)
df = df.sort_index()

print(f"Data loaded: {df.index[0].date()} to {df.index[-1].date()}")
print(f"Total rows: {len(df):,}")
print(f"\nColumns: {df.columns.tolist()}")

# Display summary
df.tail()

## 2. Configure Backtest

All parameters in one place - easy to see and modify!

In [ ]:
# Configuration dict - modify these parameters as needed
config = {
    'signal_factory': lambda: MomentumSignal(lookback=120, threshold=0.02),
    'df': df,
    'train_size': int(len(df) * 0.6),  # 60% training window
    'test_size': int(len(df) * 0.2),   # 20% testing window
    'lookback': 250,                    # History for indicators
    'initial_cash': 100_000,
    'transaction_cost': 3.0,            # 3 bps per trade
    'stop_loss_pct': 0.10,              # 10% stop loss
    'take_profit_pct': None,            # No take profit
    'max_hold_days': None,              # No max hold
    'stop_mode': 'close',               # Exit at close price
    'max_position_pct': 1.0,            # 100% invested
    'save_dir': '../logs/momentum_v1',  # Output directory
}

print("Backtest Configuration:")
print("=" * 50)
for k, v in config.items():
    if k not in ['signal_factory', 'df']:
        print(f"{k:20s}: {v}")

## 3. Run Backtest

This will:
- Run anchored walk-forward validation
- Save results to `logs/momentum_v1/`
- Auto-generate HTML report
- Print summary to console

In [ ]:
# Run backtest
results = run_walk_forward(**config)

## 4. Analyze Results

The results dict contains:
- `stitched_equity`: Portfolio value over time
- `combined_returns`: Daily strategy returns
- `trades`: All trades from all folds
- `folds`: Summary of each walk-forward fold
- `df`: Original market data

In [ ]:
# Create report object for interactive analysis
report = BacktestReport(results)

# Print summary (already shown above, but can call again)
report.summary()

## 5. Interactive Visualizations

In [ ]:
# Plot equity curve (if plotly is installed)
fig = report.plot_equity()
if fig:
    fig.show()
else:
    # Fallback to matplotlib
    plt.figure(figsize=(12, 6))
    plt.plot(results['stitched_equity'].index, results['stitched_equity'].values)
    plt.title('Equity Curve')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value ($)')
    plt.grid(True, alpha=0.3)
    plt.show()

## 6. Investigate Worst Days

In [ ]:
# Get worst 10 days with market context
worst_days = report.worst_days(10)
display(worst_days)

## 7. Investigate Worst Trades

In [ ]:
# Get worst 10 trades
worst_trades = report.worst_trades(10)
display(worst_trades)

## 8. Custom Analysis

Now you have full access to all the data for custom analysis!

In [ ]:
# Trade PnL distribution
if not results['trades'].empty:
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist(results['trades']['pnl_pct'] * 100, bins=30, alpha=0.7, edgecolor='black')
    plt.axvline(0, color='red', linestyle='--', linewidth=2, label='Breakeven')
    plt.xlabel('Trade PnL (%)')
    plt.ylabel('Frequency')
    plt.title('Trade PnL Distribution')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    cumulative_pnl = results['trades']['pnl_pct'].cumsum()
    plt.plot(cumulative_pnl.values)
    plt.xlabel('Trade Number')
    plt.ylabel('Cumulative PnL (%)')
    plt.title('Cumulative Trade PnL')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No trades to display")

In [ ]:
# Analyze trade duration
if not results['trades'].empty:
    trades = results['trades'].copy()
    trades['entry_date'] = pd.to_datetime(trades['entry_date'])
    trades['exit_date'] = pd.to_datetime(trades['exit_date'])
    trades['duration_days'] = (trades['exit_date'] - trades['entry_date']).dt.days
    
    plt.figure(figsize=(10, 5))
    plt.hist(trades['duration_days'], bins=30, alpha=0.7, edgecolor='black')
    plt.xlabel('Trade Duration (days)')
    plt.ylabel('Frequency')
    plt.title('Trade Duration Distribution')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print(f"Average trade duration: {trades['duration_days'].mean():.1f} days")
    print(f"Median trade duration: {trades['duration_days'].median():.1f} days")

In [ ]:
# Monthly returns heatmap
returns_series = results['combined_returns'].copy()
returns_series.index = pd.to_datetime(returns_series.index)

monthly = returns_series.resample('M').apply(lambda x: (1 + x).prod() - 1)
monthly_df = monthly.to_frame('return')
monthly_df['year'] = monthly_df.index.year
monthly_df['month'] = monthly_df.index.month

pivot = monthly_df.pivot(index='year', columns='month', values='return')
pivot.columns = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

plt.figure(figsize=(12, 8))
plt.imshow(pivot.values, cmap='RdYlGn', aspect='auto', vmin=-0.1, vmax=0.1)
plt.colorbar(label='Monthly Return')
plt.yticks(range(len(pivot.index)), pivot.index)
plt.xticks(range(len(pivot.columns)), pivot.columns)
plt.title('Monthly Returns Heatmap')
plt.ylabel('Year')
plt.xlabel('Month')
plt.tight_layout()
plt.show()

## 9. Walk-Forward Fold Analysis

In [ ]:
# Display fold summaries
folds_df = pd.DataFrame(results['folds'])
display(folds_df)

# Plot fold performance
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.bar(folds_df['fold'], folds_df['fold_return_pct'] * 100)
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.xlabel('Fold')
plt.ylabel('Return (%)')
plt.title('Return by Fold')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.bar(folds_df['fold'], folds_df['sharpe'])
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.xlabel('Fold')
plt.ylabel('Sharpe Ratio')
plt.title('Sharpe Ratio by Fold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Next Steps

Try modifying the configuration above:
- Change `lookback` and `threshold` parameters in MomentumSignal
- Adjust `stop_loss_pct` to see impact on risk management
- Try different transaction costs
- Experiment with different train/test splits

For other signals:
```python
from signals.mean_reversion import MeanReversionSignal
from signals.ensemble import EnsembleSignalNew

# Mean reversion
config['signal_factory'] = lambda: MeanReversionSignal(window=20, entry_z=2.0)

# Ensemble
config['signal_factory'] = lambda: EnsembleSignalNew()
```